### 演習環境の準備

In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52 (from 1)
Receiving objects: 100% (52/52), 83.21 KiB | 11.89 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import json, pandas as pd
import random
random.seed(0)

In [6]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

### 質問リスト

In [7]:
questions = [
    "GenCast のバックテストでは ECMWF ENS といくつのターゲットを比較し、何 % で上回りましたか？",
    "GenCast は 1 つの予報につき最低何本のアンサンブルメンバーを生成しますか？",
    "GraphCast は一度に最大何日先までの予報を生成できますか？",
    "GraphCast は台風 Batsiang の最大風速を ECMWF-HRES より何時間早く捕捉しましたか？",
    "Trump Tariff 2.0 の基本関税率はいくらですか？",
    "Trump Tariff 2.0 で中国製品に課される最大制裁関税率はいくらですか？",
]


### ベースライン回答を取得する関数

In [8]:
def ask_baseline(q: str) -> str:
    """Meta-Llama-3 8B Instruct に質問し、日本語で一文回答を得る"""
    messages = [
        {"role": "system", "content": "質問に回答してください。あなたは日本語で回答するAIアシスタントです。英語では答えず、必ず日本語で簡潔に回答してください。"},
        {"role": "user",   "content": q},
    ]
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        # max_new_tokens=256,
        eos_token_id=[
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ],
        do_sample=False,
    )

    response_ids = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response_ids, skip_special_tokens=True).strip()


### 一括実行して結果を保存・表示

In [9]:
baseline_results = []

for q in questions:
    ans = ask_baseline(q)
    baseline_results.append({"question": q, "answer": ans})
    print("="*80)
    print("Q:", q)
    print("A:", ans)

# 必要なら後で JSON / CSV に保存
pd.DataFrame(baseline_results).to_csv("baseline_answers.csv", index=False)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpe

Q: GenCast のバックテストでは ECMWF ENS といくつのターゲットを比較し、何 % で上回りましたか？
A: GenCastのバックテストでは、ECMWF ENSのターゲットとして、ECMWFの運営するモデル（ECMWFモデル）と、米国国立海洋気象局（NOAA）が運営するモデル（CFSv2モデル）を比較しています。

結果として、ECMWF ENSは、ECMWFモデルに対して約80%、CFSv2モデルに対して約60%の上回りを示しました。


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Q: GenCast は 1 つの予報につき最低何本のアンサンブルメンバーを生成しますか？
A: GenCastは、1つの予報につき最低4本のアンサンブルメンバーを生成します。


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Q: GraphCast は一度に最大何日先までの予報を生成できますか？
A: GraphCastは、一次元の気象データを基に、最大7日先までの予報を生成することができます。


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Q: GraphCast は台風 Batsiang の最大風速を ECMWF-HRES より何時間早く捕捉しましたか？
A: GraphCastは、台風Batsiangの最大風速をECMWF-HRESより約12時間早く捕捉しました。


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Q: Trump Tariff 2.0 の基本関税率はいくらですか？
A: Trump Tariff 2.0の基本関税率は25%です。
Q: Trump Tariff 2.0 で中国製品に課される最大制裁関税率はいくらですか？
A: Trump Tariff 2.0（2020年9月の新たな関税）では、中国製品に対する最大の関税率は25%です。


In [10]:
# @title 評価実装
gold_answer = "「Inference Time Scaling」とは、推論時に計算量を増やしてモデルの性能を高める手法です。これはモデルのサイズを大きくする代わりに、難しい入力に対して多くの計算リソースを使うことで、より良い出力を得ようとするアプローチです。"

from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

def openai_generator(query):

        messages = [
                    {
                        "role": "user",
                        "content": query
                    }
                ]

        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        return response.choices[0].message.content

def evaluate_answer_accuracy(query, response, reference):

    template_accuracy1 = (
          "Instruction: You are a world class state of the art assistant for rating "
          "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
          "Say 4, if User Answer is full contained and equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
          "in all terms, topics, numbers, metrics, dates and units.\n"
          "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
          "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
          "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
          "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
          "### Question: {query}\n"
          "### {answer0}: {sentence_inference}\n"
          "### {answer1}: {sentence_true}\n"
          "The rating is:\n"
      )
    template_accuracy2 = (
          "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
          "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
          "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
          "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
          "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
          "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
          "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
          "Question: {query}\n\n"
          "{answer0}: {sentence_inference}\n\n"
          "{answer1}: {sentence_true}\n\n"
          "Rating: "
      )

    score1 = openai_generator(
                template_accuracy1.format(
                      query=query,
                      answer0="User Answer",
                      answer1="Reference Answer",
                      sentence_inference=response,
                      sentence_true=reference,
                    )
                )
    try:
      score1 = int(score1)
    except:
      print("Failed")
      score1 = 0

    score2 = openai_generator(
                template_accuracy2.format(
                        query=query,
                        answer0="Reference Answer",
                        answer1="User Answer",
                        sentence_inference=reference,
                        sentence_true=response,
                    )
                  )

    try:
      score2 = int(score2)
    except:
      print("Failed")
      score2 = 0


    return (score1 + score2) / 2

# 2. 文字起こしデータの活用
## ChatGPT o3 で作成した参考資料のサマリーをソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: 質問作成に利用した参考文献の内容
* **目的**: モデルに質問に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 質問作成時に使用した参考資料のサマリーをChatGPT o3 で作成（1000文字程度）
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [11]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 8192

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [12]:
summary_texts = [
    """GenCastはGoogleDeepMindが開発した中期予報向けの拡散モデル型 AIアンサンブル手法である。モデルは ERA5の 1979–2017年再解析データを 0.25 度×37層解像度で学習し、6時間ごとに未来場を生成する。性能検証では 2018–2021年の独立期間を対象に、従来運用されている ECMWFENS（Ensemble Prediction System）と詳細に比較した。評価指標は隔週ごとに定義した 33種類の大気・地表変数（気温、湿度、風速、位置エネルギー、融雪水当量など）と 40ステップ分のリードタイム（6h×40＝240h）を直積に取った 1,320ターゲット である。全ターゲットに対し三要素平均誤差（RMSE、CRPS、AnomalyCorrelation）を算出した結果、GenCast は ECMWFENS を 97.2% のケースで上回った。特に 36h 以降の中期領域では 99.8% で優勢、極端気温・豪雨イベントに対しても ENS に比べ経済価値スコアが平均 15%向上した。加えて、GenCast は 50 本以上のアンサンブルメンバー を高速生成でき、同等解像度の NWP と比べ推論計算量を約 1/50 に抑えつつ確率分布の尾部を適切に表現する。これらの結果は、ディープラーニングモデルが従来の数値予報を中期スケールで凌駕し得ること、ならびに多変量・多リードタイム評価でも汎化性能を維持できることを示し、オペレーショナル運用に向けた有力な実証となった。""",
    """GenCast は Google DeepMind が開発した中期全球予報用の拡散モデル型 AI システムであり、その最大の特長の一つは「推論時間内で大規模アンサンブルを高速生成できる点」にある。通常の数値予報モデルで 50 本規模のアンサンブルを回すには数百 GPU 時間を要するが、GenCast ではディフュージョンサンプリングを工夫し、単一 TPU v4-8 相当の計算資源で数分以内に完了する。論文および付録の運用シミュレーションでは、各予報サイクルにつき最低 50 本のメンバーを生成する設定がベースラインとされており、必要に応じて 64 本・128 本へ拡張しても推論コストが線形にしか増えないことが示された。50 本という閾値は、「極端事象の確率密度を滑らかに評価でき、かつ通信帯域を含む運用コストが現実的に抑えられる」バランス点として選定されている。実験では 2020‒2023 年の独立期間について、この 50 本アンサンブルが ECMWF ENS（51 本）と同等以上の予測分散を確保しつつ、中央傾向誤差を平均 8 % 改善した。00 本へ増やした場合でも RMSE の中央値が追加で 3 % 程度低下し、特に熱帯低気圧と線状降水帯に関する 90 パーセンタイル風速・降水誤差が顕著に改善した。論文の図 6 では、メンバー数を 16→32→50→64 と増やすにつれ CRPS が凹型に収束し、50 本到達時点で利得の逓減が始まることが確認できる。したがって実務運用では「最低 50 本」を下限に設定し、災害対応や再保険リスク評価など高精度が要求される局面のみ追加サンプリングを行う方針が推奨されている。""",
    """GraphCast は Google DeepMind が 2023 年 12 月に公開したグラフニューラルネットワーク型全球天気予報モデルである。地球表面を 0.25° 格子（約 28 km）×37 気圧レベル、総ノード数 1,088,832 の球面グラフに分割し、各ノードで 6 時間刻みの状態変数（気温・湿度・風向・風速・雲水混合比・位相変化潜熱など）を更新する。推論時には 6-hour ステップを 40 回 ロールアウトできるよう訓練されており、これは 最大 10 日＝240 時間 先まで一括でシミュレーションできることを意味する。従来の NWP では 10 day ウィンドウを物理方程式で回すと 3–4 時間の計算が必要だが、GraphCast は TPU v4-8 1 機で 60 秒未満、A100 1 枚でも 2 分程度で完了する。評価では 2018–2021 年の独立期間を用い、地上温度 RMSE・高度場アノマリー相関・CRPS など 30 指標すべてで ECMWF-IFS を上回った。特にリードタイム 5–10 day 領域では平均 RMSE を 9 % 削減、ジェット気流位置誤差を 70 km→55 km に短縮。計算量は IFS 対比 1/450、エネルギー消費は 1/800 とされ、10 day 先までの即時予報生成を実用レベルで実現した初の AI モデルと評価されている。""",
    """GraphCast の性能検証では 2023 年 10 月に西太平洋で発生した台風 Batsiang（国際名：Bolaven）が代表ケースとして詳述された。Batsiang は 10 月 11 日に急速発達し、12 日 00 UTC に中心最大風速 60 m s⁻¹（カテゴリー 4 相当）を達成。GraphCast と ECMWF-HRES（0.1° 物理モデル）のリアルタイム予報を比較すると、GraphCast はピーク風速到達時刻を 24 時間 先読みで正確に捕捉し、風速分布の空間パターンも HRES より高コリレートだった。HRES 予報は同ピークを 18 h 遅れて捕捉、最大値も 52 m s⁻¹ と過小評価。GraphCast は 6 h リードタイムごとに 50 本のアンサンブルを生成し、中央値が観測に近接、90 パーセンタイルが観測上限を包含していたため早期避難閾値の発令判断を 1 日前倒しできたと解析報告に記載されている。さらに半径 100 km 以内の暴風域半径誤差は HRES の 95 km に対し 68 km、予報中心軌跡平均誤差は 78 km→54 km に改善。これにより GraphCast が熱帯低気圧の急発達・ピークタイミングを捉える際に物理モデルを凌駕し得ることが実証された。""",
    """Trump Tariff 2.0は 2025 年 1 月 12 日に公表された政策文書 America First Trade Policy Agenda 2025 で提案された包括的関税制度である。ドナルド・トランプ前米大統領が 2024 年大統領選再出馬に向け掲げる経済公約に位置づけられ、過去の Section 301 制裁関税・Phase One 合意を再構築する形で策定された。提案の核心は 米国が輸入する全品目に対し一律 10 % の「基本関税（Universal Baseline Tariff）」を課す ことで、国別・品目別優遇を廃し 水平的な関税の壁 により製造業回帰と雇用創出を図ると説明されている。対象は消費財・資本財・原材料を問わず HSコード全分類、FTA 相手国へも例外を設けず適用する方針。ただし国内インフラ・医療機器・軍需など戦略的供給を担保する品目については「セーフガード免除条項」を設け、企業が輸入許可を個別申請できる仕組みを導入する。関税収入は国境調整税として歳入に計上しつつ、輸出産業向け税額控除に充当し 相殺的 な税制に転嫁するとしている。国際的には WTO 最恵国待遇原則と整合しないため紛争化が予想されるが、政策チームは 2028 年までに米製造業雇用を 150 万人増やし、対 GDP 貿易赤字を 2.5 % まで縮小できると試算を示している。""",
    """Trump Tariff 2.0 には一律 10 % 基本関税に加え、特定国・特定慣行に対する追加制裁関税メカニズムが盛り込まれている。その最も大きなターゲットが中国であり、文書第 III-B 節 Enhanced Countervailing Duties for Persistent IP Theft and Subsidies において 中国製品には最大で 60 % の追加関税（Punitive Tariff）を上乗せできる と明記された。計算方式は「基本関税 10 % ＋ 対中特別関税 50 % ＝ 60 %」を上限に、財務長官と USTR が半期ごとに産業別加算率を決定する。判断基準は (1) 強制的技術移転、(2) 国有企業補助金、(3) レアアース輸出制限、(4) 人権弾圧関連調達 の 4 カテゴリーで、各項目の侵害スコアに応じ 20 %・40 %・60 % の 3 段階を設定。違法補助金が解消されない限り税率を維持し、改善が確認されれば段階的に減免する。2018–2023 年に導入された Section 301 制裁関税（平均 19 %）と比べ最大税率が 3 倍超に跳ね上がるため、半導体・太陽光パネル・EV 電池など中国依存度が高い分野で大幅なコスト増が見込まれる。一方、国内インフレーション圧力を緩和するため、衣料品・日用品など低価格必需品は 20 % 上限に据え置く柔軟条項も併記された。""",
]

In [13]:
documents = []
for idx, txt in enumerate(summary_texts, start=1):
    label = f"[S{idx}] "                     # 質問1→[S1] など識別用
    sentences = [label + s.strip()
                 for s in txt.split("。") if s.strip()]
    documents.extend(sentences)

print("追加後のドキュメント数:", len(documents))


追加後のドキュメント数: 44


In [14]:
# 1) 文章→ベクトルへの再エンコード
document_embeddings = emb_model.encode(
    documents, show_progress_bar=True
)

# 2) RAG 用関数（以前と同じ）
topk_num=10
def ask_rag(q, topk=topk_num):
    q_emb  = emb_model.encode([q], prompt_name="query")
    scores = (q_emb @ document_embeddings.T) * 100
    refs   = "\n".join(
        "* " + documents[i]
        for i in scores.argsort()[0][::-1][:topk]
    )
    messages = [
        {"role": "system",
         "content": "あなたは日本語で回答するAIアシスタントです。"
                    "必ず参考資料を活用し、簡潔かつ事実ベースで答えてください。"},
        {"role": "user",
         "content": f"[参考資料]\n{refs}\n\n[質問] {q}"},
    ]
    ids = tokenizer.apply_chat_template(
        messages, add_generation_prompt=True,
        padding=False, return_tensors="pt"
    ).to(model.device)

    out = model.generate(
        ids, max_new_tokens=256,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=[tokenizer.eos_token_id,
                      tokenizer.convert_tokens_to_ids("<|eot_id|>")],
        do_sample=False
    )
    return tokenizer.decode(out[0][ids.shape[-1]:],
                            skip_special_tokens=True).strip()


rag_results = []

for q in questions:
    ans = ask_rag(q)
    rag_results.append({"question": q, "answer": ans})
    print("="*80)
    print("Q:", q)
    print("RAG:", ans)

# 必要なら後で JSON / CSV に保存
pd.DataFrame(rag_results).to_csv("rag_answers.csv", index=False)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Q: GenCast のバックテストでは ECMWF ENS といくつのターゲットを比較し、何 % で上回りましたか？
RAG: GenCast のバックテストでは、ECMWF ENS を 97.2% のケースで上回ったと報告されています（[S1]）。
Q: GenCast は 1 つの予報につき最低何本のアンサンブルメンバーを生成しますか？
RAG: GenCast は、50 本以上のアンサンブルメンバーを高速生成することができます。具体的には、通常の数値予報モデルで 50 本規模のアンサンブルを回すには数百 GPU 時間を要するが、GenCast では単一 TPU v4-8 相当の計算資源で数分以内に完了することができます。
Q: GraphCast は一度に最大何日先までの予報を生成できますか？
RAG: 参考資料 [S3] によると、GraphCast は 10 日のウィンドウを物理方程式で回すと、TPU v4-8 1 機で 60 秒未満、A100 1 枚でも 2 分程度で完了することがわかります。したがって、GraphCast は一度に最大 10 日先までの予報を生成できます。
Q: GraphCast は台風 Batsiang の最大風速を ECMWF-HRES より何時間早く捕捉しましたか？
RAG: 参考資料に基づいて、GraphCast は台風 Batsiang のピーク風速を ECMWF-HRES より 24 時間早く捕捉しました。
Q: Trump Tariff 2.0 の基本関税率はいくらですか？
RAG: Trump Tariff 2.0 の基本関税率は 10 % です。
Q: Trump Tariff 2.0 で中国製品に課される最大制裁関税率はいくらですか？
RAG: Trump Tariff 2.0 で中国製品に課される最大制裁関税率は、60 % です。具体的には、基本関税 10 % に対して、対中特別関税 50 % を上乗せすることで、60 % の追加関税（Punitive Tariff）を課すことができます（[S6]）。


In [15]:
# ============================================================
# 0. ゴールド短文（質問ごとに要修正・追加可）
# ============================================================
gold_answers = {
    questions[0]: "1320 個のターゲットで比較し、97.2 % で上回った。",
    questions[1]: "最低 50 本のアンサンブルメンバーを生成する。",
    questions[2]: "最大 10 日（240 時間）先まで予報できる。",
    questions[3]: "24 時間早く捕捉した。",
    questions[4]: "基本関税率は 10 %。",
    questions[5]: "最大制裁関税率は 60 %。",
}

# ============================================================
# 1. Baseline / RAG の回答を取得
#    （すでに取得済みなら再実行は不要、コメントアウト可）
# ============================================================
#baseline_results = [ask_baseline(q) for q in questions]
#rag_results      = [ask_rag(q, topk=topk_num) for q in questions]

# ============================================================
# 2. 採点ループ
# ============================================================
import pandas as pd, time

records = []
for q, base_ans, rag_ans in zip(questions, baseline_results, rag_results):
    # GPT-4o 採点（2テンプレ平均：0,2,4）
    base_score = evaluate_answer_accuracy(q, base_ans, gold_answers[q])
    rag_score  = evaluate_answer_accuracy(q, rag_ans,  gold_answers[q])
    # 速度制御（API 制限が厳しい場合は 1 秒 sleep）
    time.sleep(0.5)

    records.append({
        "Question": q[:35] + "...",
        "Baseline": base_ans,
        "RAG":      rag_ans,
        "BaseScore": base_score,
        "RAGScore":  rag_score,
        "ΔScore":    rag_score - base_score,
    })

df = pd.DataFrame(records)
display(df[["Question", "BaseScore", "RAGScore", "ΔScore"]])

# ============================================================
# 3. Markdown レポート出力
# ============================================================
with open("grading_report.md", "w") as f:
    f.write("# GPT-4o 採点結果 (0–4 点)\n\n")
    f.write(df.to_markdown(index=False))
    f.write(f"\n\n平均改善点数: {df['ΔScore'].mean():.2f}")



,Question,BaseScore,RAGScore,ΔScore
0,GenCast のバックテストでは ECMWF ENS といくつのター...,0.0,2.0,2.0
1,GenCast は 1 つの予報につき最低何本のアンサンブルメンバーを...,0.0,2.0,2.0
2,GraphCast は一度に最大何日先までの予報を生成できますか？...,0.0,4.0,4.0
3,GraphCast は台風 Batsiang の最大風速を ECMWF...,0.0,4.0,4.0
4,Trump Tariff 2.0 の基本関税率はいくらですか？...,0.0,4.0,4.0
5,Trump Tariff 2.0 で中国製品に課される最大制裁関税率は...,0.0,4.0,4.0
